In [1]:
%load_ext autoreload
%autoreload 2

from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt

import time
from datetime import datetime
from tqdm import tqdm

In [2]:
import sys
import os
sys.path.append(os.path.dirname('../.'))
sys.path.append(os.path.dirname('../ml/.'))
from scripts.load_and_save import (get_dcm_info, get_dcm_vol, vox_size2affine,
                                   load_nii_vol, save_vol_as_nii, raw2nifti)

#from ml.get_model import get_model
from ml.utils import get_total_params, load_pretrainned, test_model
from ml.ControllerClass import Controller
from ml.tio_dataset import TioDataset
from ml.metrics import (DICE_Metric, JAC_Metric, SN_Metric, SP_Metric,
                        IOU_Metric, ExponentialLogarithmicLoss, WeightedExpBCE)

#from ml.models.HessNet import HessNet
from ml.models.unet3d import U_Net
from ml.models.unet2d import U_Net2d
from ml.models.JoB_VS import Network
from ml.transformers_models.UNETR import UNETR
from ml.models.VesselConvs import JustConv, TwoConv

from ml.models.GenUnet import GenUnet
from ml.models.HessNet_new import HessNet, GenHessNet, SeqHessNet, HessUnet

In [3]:
DEVICE = 'cuda'
IS2D = 0

In [2]:
if IS2D:
    PATCH_SIZE_TRAIN = (512, 512, 1)
    PATCH_SIZE_TEST = (512, 512, 1)
    OVERLAP_TEST = (0, 0, 0)
else:
    PATCH_SIZE_TRAIN = (64, 64, 64)
    PATCH_SIZE_TEST = (64, 64, 64)
    OVERLAP_TEST = (4, 4, 4)

train_settings  = {
    "patch_shape" : PATCH_SIZE_TRAIN,
    "patches_per_volume" : 32,
    "patches_queue_length" : 1440,
    "batch_size" : 8,
    "num_workers": 4,
    "sampler": "uniform",
}

test_settings = {
    "patch_shape" : PATCH_SIZE_TEST,
    "overlap_shape" : OVERLAP_TEST,
    "batch_size" : 1,
    "num_workers": 4,
}

data_dir = "/home/msst/Documents/medtech/data/HessData_IXI"
dataset = TioDataset(data_dir,
                 train_settings=train_settings,
                 val_settings=None,#val_settings,
                 test_settings=test_settings,)

NameError: name 'IS2D' is not defined

In [1]:
dataset

NameError: name 'dataset' is not defined

In [5]:
import torch
import torch.nn as nn

# m = HessBlock_new(in_channels=1,
#                   projection_channels=8,
#                   fc_channels=10,
#                   act=nn.Sigmoid())

# m = HessNet(in_channels=1,
#             out_channels=1,
#             n_hess_blocks=4)

# m = HessNet_(in_channels=1,
#             out_channels=1,
#             n_hess_blocks=4)

# m = GenHessNet(in_channels=1,
#             out_channels=1,
#             n_hess_blocks=4)

# m = SeqHessNet(in_channels=1,
#                out_channels=1,
#                n_hess_blocks=3)

# m = HessUnet(in_channels=1,
#              out_channels=1,
#              n_hess_blocks=4)

In [10]:
# for data in dataset.train_dataloader:
#     input = data['head']['data']
#     gt = data['vessels']['data']
#     out = m(input)
#     print(out.shape)
#     break

In [24]:
#model = HessNet(start_scale=[0.8, 0.8, 1.2], device=DEVICE)
#model = U_Net(channels=16) #Unet_8ch, Unet_16ch
#model = U_Net2d(channels=16) #Unet_8ch, Unet_16ch

# model = UNETR(in_channels=1, out_channels=1, img_size=PATCH_SIZE_TRAIN,
#               feature_size=16, hidden_size=128,
#               mlp_dim=512, num_heads=4,
#               norm_name='batch')

#model = Network(modalities=1, num_classes=1)

#model = JustConv(1, 1)
#model = TwoConv(1, 1, 5)

# model = HessBlock_new(in_channels=1,
#                       projection_channels=8,
#                       fc_channels=10,
#                       act=nn.Sigmoid())

model = HessNet(in_channels=1,
                out_channels=1,
                n_hess_blocks=4)

# model = HessNet_(in_channels=1,
#                    out_channels=1,
#                    n_hess_blocks=4)

# model = GenHessNet(in_channels=1,
#                    out_channels=1,
#                    n_hess_blocks=4)

# model = SeqHessNet(in_channels=1,
#                    out_channels=1,
#                    n_hess_blocks=3)

# model = HessUnet(in_channels=1,
#                  out_channels=1,
#                  n_hess_blocks=4)

# model = GenUnet(dim=3, in_channels=1, out_channels=1,
#                channels=8, depth=1)

print("total_params:", get_total_params(model))

total_params: 6630


In [25]:
controller_config = {
    # "loss" : ExponentialLogarithmicLoss(gamma_tversky=0.5, gamma_bce=0.5, lamb=0.5,
    #                                     freq = 0.1, tversky_alfa=0.5),
    "loss" : ExponentialLogarithmicLoss(gamma_tversky=0.5, gamma_bce=0.5, lamb=0.1,
                                        freq = 0.1, tversky_alfa=0.2),

    "metric" : DICE_Metric(),
    'device' : DEVICE,
    "model" : model,
    "optimizer_fn" : lambda model: Adam(model.parameters(), lr=0.02),
    "sheduler_fn": None, #lambda optimizer: StepLR(optimizer, step_size=1, gamma=0.9),
    "is2d" : IS2D,
    'verbose' : True,
    'stoper' : None
}
controller = Controller(controller_config)

In [13]:
controller.fit(dataset, 15)

Epoch 1/15


100%|███████████████████████████████████████████| 72/72 [00:26<00:00,  2.71it/s]


{'mean_loss': 1.024795574032598}


100%|█████████████████████████████████████████████| 2/2 [00:04<00:00,  2.28s/it]


{'sample': 'IXI300_0', 'metric': tensor([0.3019])}
{'sample': 'IXI020_0', 'metric': tensor([0.4437])}
new best!
Epoch 2/15


...

100%|███████████████████████████████████████████| 72/72 [00:26<00:00,  2.72it/s]


{'mean_loss': 0.41379570836822194}


100%|█████████████████████████████████████████████| 2/2 [00:04<00:00,  2.35s/it]

{'sample': 'IXI300_0', 'metric': tensor([0.8600])}
{'sample': 'IXI020_0', 'metric': tensor([0.8610])}
count_without_new_best_test_val: 1


U_Net(
  (input_noise): Identity()
  (noise_e2): Identity()
  (Maxpool1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Maxpool2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Maxpool3): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Maxpool4): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv1): conv_block(
    (conv): Sequential(
      (0): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (4): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (Conv2): conv_block(
    (conv): Sequential(
      (0): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), 

In [26]:
#model_name = "Unet3d_16ch_21.03_2"
#model_name = "Unet2d_16ch"
#model_name = "HessNet_isotropic_smartnorm"
#model_name = "UNETR"

#model_name = "HessBlock_proj8"
#model_name = 'HessNet_new'
#model_name = 'GenHessNet'
#model_name = 'SeqHessNet'
#model_name = 'HessNet_new_'
#model_name = 'HessUnet'

model_name = 'HessNet_new_'
#model_name = "LittleUnet"

In [27]:
#controller.save(f"/home/msst/save_folder/saved_models/{model_name}.pth")
#controller.save_weights(f"/home/msst/save_folder/saved_models/{model_name}_weights.pth")

In [28]:
path_to_check = f"/home/msst/save_folder/saved_models/{model_name}.pth"
controller.load(path_to_checkpoint=path_to_check)

In [29]:
from scripts.utils import get_path
import torchio as tio

#sample_index = "092"
#sample_index = "200"
#sample_index = "131"
sample_index = "300"

path_to_vol = get_path(f"/home/msst/IXI_MRA_work/IXI{sample_index}", key="head")
subject_dict = {'head': tio.ScalarImage(path_to_vol)}
subject = tio.Subject(subject_dict)
subject = tio.transforms.ZNormalization()(subject)

t = time.time()
seg = controller.single_predict(subject, test_settings, thres=None)
print(time.time() - t)

#dt_string = datetime.now().strftime("%d_%m_%Y_%H:%M")
dir_name = os.path.dirname(path_to_vol)

#path_to_save = f"/home/msst/IXI_MRA_work/IXI{sample_index}/{model_name}_{dt_string}.nii.gz"
path_to_save = f"/home/msst/IXI_MRA_work/IXI{sample_index}/{model_name}.nii.gz"
save_vol_as_nii(seg, subject.head.affine, path_to_save)

3.072436809539795


In [31]:
path_to_GT = get_path(f"/home/msst/IXI_MRA_work/IXI{sample_index}", key="vessels")
GT = tio.ScalarImage(path_to_GT).data
path_to_brain_mask = get_path(f"/home/msst/IXI_MRA_work/IXI{sample_index}", key="brain")
brain_mask = tio.ScalarImage(path_to_brain_mask).data

path_to_save_masked = f"/home/msst/IXI_MRA_work/IXI{sample_index}/{model_name}_mask.nii.gz"
path_to_save_GT_masked = f"/home/msst/IXI_MRA_work/IXI{sample_index}/vessels_mask.nii.gz"
save_vol_as_nii(seg*brain_mask, subject.head.affine, path_to_save_masked)
save_vol_as_nii(GT*brain_mask, subject.head.affine, path_to_save_GT_masked)

IndexError: list index out of range

In [30]:
import subprocess
import re

path_to_EvaluateSegmentation = '/home/msst/repo/MSRepo/VesselSegmentation/Inference/EvaluateSegmentation'

GT_path = path_to_GT
SEG_path = path_to_save
GT_mask_path = path_to_save_masked
SEG_mask_path = path_to_save_GT_masked

command_output = subprocess.run([f"{path_to_EvaluateSegmentation}",
                                    GT_path, SEG_path], stdout=subprocess.PIPE, text=True)
command_output = command_output.stdout.split('\n')
metrics = ["DICE", "AVGDIST", "SNSVTY"]
metric_dict = {}
for metric in metrics:
    for line in command_output:
        if re.search(metric, line):
            metric_dict.update({metric : line.split('\t')[1][2:]})

command_output = subprocess.run([f"{path_to_EvaluateSegmentation}",
                                    GT_mask_path, SEG_mask_path], stdout=subprocess.PIPE, text=True)
command_output = command_output.stdout.split('\n')
metrics = ["DICE", "AVGDIST", "SNSVTY"]
metric_dict_mask = {}
for metric in metrics:
    for line in command_output:
        if re.search(metric, line):
            metric_dict_mask.update({metric : line.split('\t')[1][2:]})

metric_dict, metric_dict_mask

NameError: name 'path_to_GT' is not defined

**<h3>GLOBAL TEST**

In [3]:
def run_tests(model_name, path_to_save_models, is2d, test_data_path, epochs):
    if is2d:
        PATCH_SIZE_TRAIN = (512, 512, 1)
        PATCH_SIZE_TEST = (512, 512, 1)
        OVERLAP_TEST = (0, 0, 0)
    else:
        PATCH_SIZE_TRAIN = (64, 64, 64)
        PATCH_SIZE_TEST = (256, 256, 64)
        OVERLAP_TEST = (32, 32, 24)
    
    train_settings  = {
        "patch_shape" : PATCH_SIZE_TRAIN,
        "patches_per_volume" : 64,
        "patches_queue_length" : 1440,
        "batch_size" : 16,
        "num_workers": 4,
        "sampler": "uniform",
    }
    
    test_settings = {
        "patch_shape" : PATCH_SIZE_TEST,
        "overlap_shape" : OVERLAP_TEST,
        "batch_size" : 1,
        "num_workers": 4,
    }
    
    for test in tqdm(os.listdir(test_data_path)):
        dataset = TioDataset(test_data_path + '/' + test,
                  train_settings=train_settings,
                  test_settings=test_settings)
        
        model = get_model(model_name)
        
        controller_config = {
            "loss" : ExponentialLogarithmicLoss(gamma_tversky=0.5, gamma_bce=0.5, lamb=0.5,
                                                freq = 0.1, tversky_alfa=0.5),
            "metric" : DICE_Metric(),
            'device' : 'cuda',
            "model" : model,
            "optimizer_fn" : lambda model: torch.optim.Adam(model.parameters(), lr=0.02),
            "sheduler_fn": lambda optimizer: StepLR(optimizer, step_size=1, gamma=0.9),
            "is2d" : is2d,
            'verbose': False
        }
        controller = Controller(controller_config)
        controller.fit(dataset, epochs)
        controller.save(f"{path_to_save_models}/{model_name}_{test}")

In [4]:
#model_name = 'HessUNet2'
model_name = 'Unet3d_16ch'

run_tests(model_name=model_name, 
          path_to_save_models="/home/msst/save_folder/models_for_tests",
          test_data_path = '/home/msst/Documents/medtech/data/HessData_IXI_test',
          is2d=False,
          epochs=25)

  0%|                                                     | 0/2 [00:00<?, ?it/s]

Epoch 1/25
{'mean_loss': 0.7622763651112715}
{'metrics': [{'sample': 'IXI111_0', 'metric1': tensor([0.7776])}, {'sample': 'IXI100_0', 'metric1': tensor([0.7168])}, {'sample': 'IXI020_0', 'metric1': tensor([0.7513])}]}
Epoch 2/25
{'mean_loss': 0.2930100342879693}
{'metrics': [{'sample': 'IXI111_0', 'metric1': tensor([0.8368])}, {'sample': 'IXI100_0', 'metric1': tensor([0.7817])}, {'sample': 'IXI020_0', 'metric1': tensor([0.8290])}]}
Epoch 3/25
{'mean_loss': 0.2731123532479008}
{'metrics': [{'sample': 'IXI111_0', 'metric1': tensor([0.8429])}, {'sample': 'IXI100_0', 'metric1': tensor([0.7831])}, {'sample': 'IXI020_0', 'metric1': tensor([0.8305])}]}
Epoch 4/25
{'mean_loss': 0.26982324353108805}
{'metrics': [{'sample': 'IXI111_0', 'metric1': tensor([0.8570])}, {'sample': 'IXI100_0', 'metric1': tensor([0.7953])}, {'sample': 'IXI020_0', 'metric1': tensor([0.8501])}]}
Epoch 5/25
{'mean_loss': 0.25067211718608934}
{'metrics': [{'sample': 'IXI111_0', 'metric1': tensor([0.8659])}, {'sample': 'IXI

 50%|█████████████████████▌                     | 1/2 [18:16<18:16, 1096.89s/it]

{'metrics': [{'sample': 'IXI111_0', 'metric1': tensor([0.9343])}, {'sample': 'IXI100_0', 'metric1': tensor([0.8491])}, {'sample': 'IXI020_0', 'metric1': tensor([0.9339])}]}
Epoch 1/25
{'mean_loss': 0.6672985454400381}
{'metrics': [{'sample': 'IXI077_0', 'metric1': tensor([0.3137])}, {'sample': 'IXI111_0', 'metric1': tensor([0.3520])}, {'sample': 'IXI115_0', 'metric1': tensor([0.4546])}]}
Epoch 2/25
{'mean_loss': 0.2853224550684293}
{'metrics': [{'sample': 'IXI077_0', 'metric1': tensor([0.7835])}, {'sample': 'IXI111_0', 'metric1': tensor([0.8429])}, {'sample': 'IXI115_0', 'metric1': tensor([0.7320])}]}
Epoch 3/25
{'mean_loss': 0.26316667441278696}
{'metrics': [{'sample': 'IXI077_0', 'metric1': tensor([0.7689])}, {'sample': 'IXI111_0', 'metric1': tensor([0.8603])}, {'sample': 'IXI115_0', 'metric1': tensor([0.7597])}]}
Epoch 4/25
{'mean_loss': 0.2678287575642268}
{'metrics': [{'sample': 'IXI077_0', 'metric1': tensor([0.7987])}, {'sample': 'IXI111_0', 'metric1': tensor([0.8774])}, {'sample

100%|███████████████████████████████████████████| 2/2 [36:29<00:00, 1094.53s/it]

{'metrics': [{'sample': 'IXI077_0', 'metric1': tensor([0.8792])}, {'sample': 'IXI111_0', 'metric1': tensor([0.9370])}, {'sample': 'IXI115_0', 'metric1': tensor([0.8047])}]}
